In [1]:
import torch

In [2]:
!cp drive/MyDrive/Research_Fisheye/fisheye_detection/LaRecNet/* .


cp: -r not specified; omitting directory 'drive/MyDrive/Research_Fisheye/fisheye_detection/LaRecNet/__pycache__'


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.empty_cache()

In [7]:
DRIVE_PATH  = "/content/drive/MyDrive/Research_Fisheye/"

In [8]:
# !ls drive/MyDrive/Research_Fisheye/Datasets/wireframe/fisheye_pointlines/ > ./current.txt

In [9]:
# !cp ./current.txt drive/MyDrive/Research_Fisheye/fisheye_detection/LaRecNet/current.txt

In [10]:
from larecnet import LaRecNet
from resnet import BasicBlock
import torch.nn as nn
import torch.nn.parallel
import torch.optim
from datasets import *
from torch.utils.data import DataLoader

In [11]:
DATASET = "wireframe"
DATASET_PATH = DRIVE_PATH + "Datasets/wireframe/"
MODEL_PATH = "Models/"
EPOCHS = 100
LR = 0.00001
BATCH_SIZE = 8
LOSSES = []

In [12]:
def r_f(angle, k, num_params=5):
    result = 0
    for i in range(num_params):
        result += k[i] * angle ** (2 * (i + 1) - 1)

    return result

In [13]:
class LaRecNetLoss(nn.Module):
    def __init__(self, weights=[], lambda_fus=2, lambda_global=1, lambda_local=1,
                 lambda_m=2, lambda_geo=100, lambda_pix=1, lambda_para=1):
        super(LaRecNetLoss, self).__init__()
        # params for MCM
        self.weights = np.ones(9)
        self.lambda_fus = lambda_fus
        self.lambda_global = lambda_global
        self.lambda_local = lambda_local

        # params for overall network
        self.lambda_m = lambda_m
        self.lambda_geo = lambda_geo
        self.lambda_pix = lambda_pix
        self.lambda_para = lambda_para

        # fix focal length: True ? False
        self.fix_focal = False
        self.focal = 0

    def fix_focal_length(self, f):
        self.fix_focal = True
        self.focal = f

    def forward(self, x, gt):
        k_local, k_global, k_hat = x[0], x[1], x[2]
        loss_global = 1 / 9 * (np.dot(self.weights, (k_global, gt["distortion"]))) ** 2
        loss_local = 1 / 25 * (self.weights[0:5] * np.dot(k_local, gt["distortion"][0:5])) ** 2
        loss_fused = 1 / 9 * (np.dot(self.weights, (k_hat, gt["distortion"]))) ** 2

        # loss_para = self.lambda_fus * loss_fused + self.lambda_global * loss_global + self.lambda_local * loss_local
        #
        # geometric_err = 0
        # image_size = len(gt["img"][0])

        return loss_fused + loss_local + loss_global


In [14]:
def train(model, inputs, ground_truth):
    # loss_func = LaRecNetLoss(weights)
    loss_func = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    total_loss = 0
    inputs = torch.reshape(inputs, (BATCH_SIZE, 3, 320, 320))
    prediction = model(inputs)
    # print("Model Output:", prediction)
    loss = loss_func(prediction, ground_truth)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    return total_loss

In [15]:
def main():
    if DATASET == "wireframe":
        wireframe = Wireframe(DATASET_PATH + "v1.1/train_1024.txt")
        dataset_loader = DataLoader(dataset=wireframe, batch_size=BATCH_SIZE, shuffle=True)
    else:
        dataset_loader = None

    model = LaRecNet(block=BasicBlock, layers=[2, 2, 2, 2], batch_size=BATCH_SIZE)
    model.to(device)
    # loss_func = LaRecNetLoss(weights)
    loss_func = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [30])

    for i in range(EPOCHS):
        epoch_loss = []
        for idx, data in enumerate(dataset_loader):
            inputs, ground_truth = data
            inputs = torch.reshape(inputs, (BATCH_SIZE, 3, 320, 320))
            inputs = inputs.to(device)
            ground_truth = ground_truth.to(device)
            prediction = model(inputs)
            print("F_GroundTruth:", torch.reshape(ground_truth ,(1, BATCH_SIZE)))
            loss = loss_func(prediction, ground_truth)
            epoch_loss.append(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # scheduler.step()
            print()
        avg_loss = sum(epoch_loss) / len(epoch_loss)
        LOSSES.append(avg_loss)
        torch.save(model, DRIVE_PATH + MODEL_PATH + "EPOCH %d.pkl" %i)
        print("\n----------------------------------")
        print(LOSSES[-5:])
        print("EPOCH %d FINISH" % i)
        print("\n----------------------------------")
    return

In [ ]:
main()